In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Orçamento\\ONE DRIVE ORCAMENTO\\OneDrive - GRUPO RETEC\\02. Engenharia\\IA\\ORCA\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\Orçamento\\ONE DRIVE ORCAMENTO\\OneDrive - GRUPO RETEC\\02. Engenharia\\IA\\ORCA'

# === ENTITY ===

In [38]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    learning_rate: float
    n_estimators: float
    target_column: float

# === CONFIGURATION MANAGER ===

In [39]:
from ORCA.constants import *
from ORCA.utils.common import read_yaml, create_directories

In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBoosting
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            learning_rate = params.learning_rate,
            n_estimators = params.n_estimators,
            target_column = schema.name
            
        )

        return model_trainer_config

# === COMPONENT ===

In [41]:
import pandas as pd
import os
from ORCA import logger
from sklearn.linear_model import ElasticNet
import joblib

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.tools import add_constant
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import cross_val_score
import joblib
import optuna

In [45]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_excel(self.config.train_data_path)
        test_data = pd.read_excel(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        y_train_log = np.log1p(train_y)
        y_test_log = np.log1p(test_y)

        model = XGBRegressor(
            learning_rate=self.config.learning_rate,
            max_depth=5,
            n_estimators=self.config.n_estimators,
            subsample=0.8,
            colsample_bytree=1,
            colsample_bynode=0.75,
            gamma=0,
            min_child_weight=1,
            reg_alpha=0.1,
            reg_lambda=1,
            objective='reg:squarederror'
        )
        model.fit(train_x, y_train_log)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))



In [46]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-07-25 16:10:47,696: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-25 16:10:47,701: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 16:10:47,710: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-25 16:10:47,713: INFO: common: created directory at: artifacts]
[2025-07-25 16:10:47,717: INFO: common: created directory at: artifacts/model_trainer]
